In [ ]:
import os
import numpy as np
import torch as T
import torch.nn.functional as F
from memory_buffer import ReplyBuffer
from networks import ActorNetwork, ValueNetwork, CriticNetwork

class Agent():
    def __init__(self, input_dims, alpha=0.003, beta = 0.003, env = None, gamma = 0.99, n_actions = 2, memory_size = 1000000, tau = 0.005, layer1 = 256 , layer2 = 256, batch_size = 256, reward_scale = 2): 
        # reward scaling is how are we going to account for the entropy in the framework - we scale the rewards in the critic loss function
    
        self.gamma = gamma
        self.tau = tau
        self.batch_size = batch_size
        self.n_actions = n_actions
        
        self.memory = ReplyBuffer(memory_size, input_dims, n_actions)
        self.actor = ActorNetwork(alpha, input_dims, max_action = env.action_space.high)
        self.critic1 = CriticNetwork(beta, input_dims, n_actions, layer1, layer2, name='critic1')
        self.critic2 = CriticNetwork(beta, input_dims, n_actions, layer1, layer2, name='critic2')
        self.val = ValueNetwork(beta, input_dims, layer1, layer2, name='value')
        self.target_val = ValueNetwork(beta, input_dims, layer1, layer2, name='target_val')

        self.scale = reward_scale
        self.update_pars(tau = 1) # come back to this!

        def pick_action(self, observation):
            state = T.Tensor([observation]).to(self.actor.device) # convert observation to pythorch sensor and send it to the device
            action, _ = self.actor.normal_sample(state, reparametrize = False) # we dont include the noise
            # here action is array of actions cuz we are dealing with a continuous action space
            return action.cpu().detach().numpy()[0] # we extract and the selected action as np array
        
        def remember(self, state, action, reward, next_state, done):
            self.memory.store_transition(self, state, action, reward, next_state, done)

        # The update_pars method is used to perform a soft update of the target value network parameters 
        # This is done by taking a weighted average of the current value network parameters and the target value network parameters, 
        # where the weight is determined by the parameter tau. This method helps to stabilize learning by having a slowly changing 
        # target for the value network to learn from.
        def update_pars(self, tau = None):
            # at the beginning of the simulation we want to set the values for the target network to an 
            # exact copy of the value network so the target value should be an exact copy of the value network,
            # but on every other step we want it to be a soft copy
            if tau == None:
                tau = self.tau
            
            target_val_pars = self.target_val.named_parameters()
            val_pars = self.val.named_parameters()

            target_val_state_dict = dict(target_val_pars)
            val_state_dict = dict(val_pars)

            for name in val_state_dict:




    


        